##  We're now going to reat the turnout data and analyze turnout by precinct for only 2008

In [1]:
#First set everthing up
import pyodbc
import graphlab as gl
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime
import local #server information and credentials
from pullODBC import pullODBC
from pullODBCall import pullODBCall
from matplotlib.backends.backend_pdf import PdfPages
from __future__ import division # convert integer division to floating point
%matplotlib inline
plt.style.use('seaborn-white')
plt.rcParams['font.size'] = 7
plt.tight_layout;
plt.rc('xtick', labelsize=7) 
plt.rc('ytick', labelsize=7) 

### Section A: Analyzing Turnout by Precinct for Voters Affiliated with a Party

Read in the csv for the combined data: county, precinct, party code, and 2106 turnout

In [2]:
combined = pd.read_csv('C:/voter/combined2008.csv', low_memory=True)

C:\Users\Scott\Anaconda2\envs\gl-env\lib\site-packages\IPython\core\interactiveshell.py:2723: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
print len(combined)

6264627


In [4]:
print combined['2008'].sum()

4308287.0


Print out the columns of this dataframe

In [5]:
print combined.columns

Index([u'Unnamed: 0', u'birth_age', u'gender_code', u'race_code',
       u'ethnic_code', u'party_cd', u'status_cd', u'zip_code', u'birth_state',
       u'county_desc', u'registr_dt', u'ncid', u'precinct_abbrv',
       u'Unnamed: 0.1', u'2007', u'2008', u'2009', u'2010', u'2011', u'2012',
       u'2013', u'2014', u'2015', u'2016', u'2018'],
      dtype='object')


Subset the dataframe to leave only the relevent columns 

In [6]:
turnout = combined.ix[:,['county_desc', 'party_cd', 'precinct_abbrv', '2008']]
del combined

In [7]:
print turnout.head()

  county_desc party_cd precinct_abbrv  2008
0     CHATHAM      DEM         45       1.0
1   CLEVELAND      DEM         KM 2     1.0
2    COLUMBUS      DEM         P01A     1.0
3      CRAVEN      DEM         14       1.0
4  CUMBERLAND      REP         G9A      0.0


Creat a custom aggfunc to count the percentage of rows with '1' in column '2016'

In [8]:
def percent_non_zero(x):
    frac = np.sum(x)/np.size(x)
    return(frac)

Create a Pivot Table with index of county, precinct and columns party_cd and aggregation 2016

In [9]:
registered_by_precinct = pd.pivot_table(turnout, index=['county_desc','precinct_abbrv'], values=['2008'],  
                                 aggfunc=percent_non_zero, columns=['party_cd'], margins = False) 


In [10]:
pd.options.display.float_format = '{:.1%}'.format #display as percentages
new_dataframe = pd.DataFrame.from_records(registered_by_precinct.to_records())
new_dataframe.columns




Index([u'county_desc', u'precinct_abbrv', u'('2008', 'DEM')',
       u'('2008', 'LIB')', u'('2008', 'REP')', u'('2008', 'UNA')'],
      dtype='object')

In [11]:
new_dataframe.to_csv('C:/voter/turnout2008.csv')

In [12]:
print new_dataframe.head()

  county_desc precinct_abbrv  ('2008', 'DEM')  ('2008', 'LIB')  \
0    ALAMANCE         01                68.3%           100.0%   
1    ALAMANCE         02                72.1%           100.0%   
2    ALAMANCE         035               73.6%           100.0%   
3    ALAMANCE         03C               82.5%             nan%   
4    ALAMANCE         03N               76.7%            75.0%   

   ('2008', 'REP')  ('2008', 'UNA')  
0            74.5%            60.2%  
1            80.5%            70.3%  
2            77.8%            65.8%  
3            83.8%            72.3%  
4            78.5%            71.0%  
